In [2]:
from operator import *
import itertools as it
import numpy as np
import time
import copy
import os

In [3]:
# Testing how to do Cartesian products of lists
# This will be needed later to stitch together lists of sides to get a full corona
A=[[1]]
B=[[2]]
C=[[3]]
D=[[4],[5]]
X = it.product(A,B,C,D)
for i in X:
    print(i)

([1], [2], [3], [4])
([1], [2], [3], [5])


# Side Type Generators

In [4]:
# C = size of central square
# m = maximum size square (square sizes are 1, 2, ..., m)
# A side contains squares of sizes a0, a1, ..., ak
# A side is denoted by a0^e.a1.a2. ... . ak where the exponent e on a0 is the amount of "prehang" 
# that the first square a0 has with the central square.
# In general, e satisfies 0 <= e <= a0.
# The data type for a side is [e,[a0,a1,...,ak]]

## Side Type i

In [72]:
# Side type i (afterhang)
# Type i has the following constraints:

# exponent constraint: 
# e = 0
 
# Partition Constraints:
# Because the last square ak must overhang, we must have
# C + 1 <= sum(ai,(i,0,k)) <= (C + m) - 1
# This tells us that we want to find integer partitions of 
# C+1, C+2, ..., and (C + m) - 1 to get the side sequences a0,a1,...,ak

# Overlap Constraints:
# The last square ak must share a nonzero edge length with the central square C,
# but ak should not perfectly align with the corner of the central square C
# This gives rise to the numerical constraint on the sides:
# sum(ai,(i,0,k-1)) < C < sum(ai,(i,0,k))
# This will serve as a filter to weed out partitions a0,a1,...,ak that don't overhang properly

# Other filters:
# (1) Remove instances where a0 = C (violates unilaterality since e = 0 in this case)
# (2) Remove instances where ai = a(i+1)
# (3) Remove instances where the partition contains "a,1,b" or "a,2,b" (violates vortex condition for 1- and 2-squares)
# (4) Apply overlap conditions as a filter (these require summing so should come last)

# Outline of algorithm:
# inputs: C = size of center square, m = max size square
# For each value of n = C+1, C+2, ... (C + m) - 1:
#     Let P(n) = List of all integer partitions of n (and their distinct permutations)
#     where the partitions use only integers 1, 2, ..., m
#     Let FilteredP(n) = P(n)
#     For each L in P(n):
#         Apply various filters to detect if L should be removed from P(n).
#             If L fails a filter:
#                 FilteredP(n).remove(L)
#
#     return FilteredP(n)

def Side_Type_i_Generator(C,m):

    #Using integer partitions to create a list of all potential sides of the form [0,[a0,a1,...,ak]]
    AllPotentialSides = []
    for n in range(C+1,C+m):
        
        #Generate all partitions of n using integers 1, 2, ..., m
        P = Partitions(n,max_part = m).list()
        #print(P)
        
        #Creating all permutations of the integer partitions
        for BasePart in P:
            MoreParts = Permutations(BasePart) 
            for x in MoreParts:
                AllPotentialSides.append([0,x]) #the exponent is 0 for all sides of type 
                
       
    # Filtered Sides is intialized as a copy of AllPotentialSides.
    # We will remove sides from FilteredSides when the are caught by a filter.
    FilteredSides = []
    for i in AllPotentialSides:
        FilteredSides.append(i)
    
    for L in AllPotentialSides:

        #print("\n L = ",L)

        #Remove any instances where a0 = C
        if L[1][0] == C:
            FilteredSides.remove(L)
            #print("This L got filtered since the first square equals the center (unilaterality check)")
            continue
            
        #Remove any instances where L has ai = a(i+1) for some i = 0, ..., n-2
        flag = False
        for i in range(len(L[1])-1):
            if L[1][i] == L[1][i+1]:
                flag = True
                break
        if flag == True:
            FilteredSides.remove(L)
            #print("This L got filtered since the consecutive sides are equal")
            continue

        #Remove any instances where L contains "a,1,b" or "a,2,b" since any such instance
        #violates the vortex condition for size-1 and size-2 squares
        flag = False
        for i in range(1,len(L[1])-1):
            if (L[1][i] == 1) or (L[1][i] == 2):
                flag = True
                break
        if flag == True:
            FilteredSides.remove(L)
            #print("This L got filtered since there was a 1 or 2 in the middle")
            continue

        # Using the overlap constraints to filter out partitions
        # We must have sum(ai,(i,0,k-1)) < C < sum(ai,(i,0,k))
        # Remove any L that does not satisfy that condition
        
        Lsum1 = sum(L[1][i] for i in range(len(L[1])-1))
        Lsum2 = sum(L[1][i] for i in range(len(L[1])))
        #print("Lsum1 = ", Lsum1)
        #print("Lsum2 = ", Lsum2)
  
        if Lsum1 >= C or C >= Lsum2:
            FilteredSides.remove(L)
            #print("This L got filtered by overlap constraints")
            continue
        #else:
        #    print("This L survived")
        #    print("C = ", C)
        #    print("Lsum1 = ", Lsum1)
        #    print("Lsum2 = ", Lsum2)
    
    #print(FilteredSides)
    
    return(FilteredSides)

In [73]:
Side_Type_i_Generator(7,5)

[[0, [5, 3]],
 [0, [3, 5]],
 [0, [1, 5, 2]],
 [0, [1, 4, 3]],
 [0, [1, 3, 4]],
 [0, [2, 4, 2]],
 [0, [5, 4]],
 [0, [4, 5]],
 [0, [1, 5, 3]],
 [0, [1, 3, 5]],
 [0, [2, 4, 3]],
 [0, [2, 3, 4]],
 [0, [1, 5, 4]],
 [0, [1, 4, 5]],
 [0, [2, 3, 5]],
 [0, [2, 4, 5]]]

## Side Type ii

In [70]:
# Side type ii (prehang)
# Type ii has the following constraints:

# exponent constrait: 
# 0 < e <= a0 and e is an integer

# partition constraint:
# Because the first square a0 must overhang, we must have
# C + 1 <= sum(ai,(i,0,k)) <= (C + m)
# This tells us that we want to find integer partitions of 
# C+1, C+2, ..., and (C + m) to get the side sequences a0,a1,...,ak 

# overlap constraint 1:
# e = sum(ai,(i,0,k)) - C
# This determines the exponent e and we can check against
# the exponent constraint to filter out bad partitions a0,a1,...,ak

# overlap constraint 2:
# sum(ai,(i,1,k)) <= C < sum(ai,(i,0,k))
# This can be used to filter out partitions that cannot overlap properly
# at the a0 square.

# Other filters:
# (1) Remove instances where ak = C (violates unilaterality since e = 0 in this case)
# (2) Remove instances where ai = a(i+1)
# (3) Remove instances where the partition contains "a,1,b" or "a,2,b" (violates vortex condition for 1- and 2-squares)
# (4) Apply overlap conditions as a filter (these require summing so should come last)

def Side_Type_ii_Generator(C,m):

    # Using integer partitions to create a list of all potential sides of the form [e,[a0,a1,...,ak]]
    # The partition constraint tells us to find partitions of C+1, C+2, ..., and (C + m)
    AllPotentialSides = []
    for n in range(C+1,C+m+1):
        
        #Generate all partitions of n using integers 1, 2, ..., m
        P = Partitions(n,max_part = m).list()
        #print(P)
        
        #Creating all permutations of the integer partitions
        for BasePart in P:
            MoreParts = Permutations(BasePart) 
            # Adding exponents
            for x in MoreParts:
                # The exponent constraint is 0 < e <= a0 and e is an integer
                a0 = x[0]
                for e in range(1,a0+1):
                    AllPotentialSides.append([e,x])
                
       
    # Filtered Sides is intialized as a copy of AllPotentialSides.
    # We will remove sides from FilteredSides when the are caught by a filter.
    FilteredSides = []
    for i in AllPotentialSides:
        FilteredSides.append(i)
    
    for L in AllPotentialSides:

        #print("\n L = ",L)

        #Remove any instances where ak = C
        if L[1][len(L[1])-1] == C:
            FilteredSides.remove(L)
            #print("This L got filtered since the last square equals the center (unilaterality check)")
            continue
            
        #Remove any instances where L has ai = a(i+1) for some i = 0, ..., n-2
        flag = False
        for i in range(len(L[1])-1):
            if L[1][i] == L[1][i+1]:
                flag = True
                break
        if flag == True:
            FilteredSides.remove(L)
            #print("This L got filtered since the consecutive sides are equal")
            continue

        #Remove any instances where L contains "a,1,b" or "a,2,b" since any such instance
        #violates the vortex condition for size-1 and size-2 squares
        flag = False
        for i in range(1,len(L[1])-1):
            if (L[1][i] == 1) or (L[1][i] == 2):
                flag = True
                break
        if flag == True:
            FilteredSides.remove(L)
            #print("This L got filtered since there was a 1 or 2 in the middle")
            continue

        # Using the overlap constraints to filter out partitions.
        # Constraint 1: e = sum(ai,(i,0,k)) - C
        # Constraint 2: sum(ai,(i,1,k)) <= C < sum(ai,(i,0,k))
        # Remove any L that does not satisfy these conditions
        
        Lsum1 = sum(L[1][i] for i in range(1,len(L[1])))
        Lsum2 = sum(L[1][i] for i in range(len(L[1])))
        #print("Lsum1 = ", Lsum1)
        #print("Lsum2 = ", Lsum2)

        # Checking overlap constraint #1:
        if L[0] != (Lsum2 - C):
            FilteredSides.remove(L)
            #print("This L got filtered by overlap constraint #1")
            continue

        # Checking overlap constraint #2
        if Lsum1 > C or C >= Lsum2:
            FilteredSides.remove(L)
            #print("This L got filtered by overlap constraint #2")
            continue
        #else:
        #    print("This L survived")
        #    print("C = ", C)
        #    print("Lsum1 = ", Lsum1)
        #    print("Lsum2 = ", Lsum2)
    
    #print(FilteredSides)
    
    return(FilteredSides)


In [71]:
Side_Type_ii_Generator(7,5)

[[1, [5, 3]],
 [1, [3, 5]],
 [1, [2, 5, 1]],
 [1, [1, 5, 2]],
 [1, [4, 3, 1]],
 [1, [3, 4, 1]],
 [1, [1, 4, 3]],
 [1, [1, 3, 4]],
 [1, [2, 4, 2]],
 [2, [5, 4]],
 [2, [4, 5]],
 [2, [5, 3, 1]],
 [2, [3, 5, 1]],
 [2, [2, 5, 2]],
 [2, [4, 3, 2]],
 [2, [3, 4, 2]],
 [2, [2, 4, 3]],
 [2, [2, 3, 4]],
 [3, [5, 4, 1]],
 [3, [4, 5, 1]],
 [3, [5, 3, 2]],
 [3, [3, 5, 2]],
 [3, [3, 4, 3]],
 [4, [5, 4, 2]],
 [4, [4, 5, 2]],
 [4, [4, 3, 4]],
 [5, [5, 4, 3]],
 [5, [5, 3, 4]]]

## Side Type iii

In [54]:
# Side type iii (nohang)
# Type iii has the following constraints:

# exponent constrait: e = 0.

# Partition constraint:
# sum(ai,(i,0,k)) = C
# We must simply find all partitions of C

def Side_Type_iii_Generator(C,m):

    # Using integer partitions to create a list of all potential sides of the form [e,[a0,a1,...,ak]]
    # The partition constraint tells us to find partitions of C.
    AllPotentialSides = []
           
    #Generate all partitions of C using integers 1, 2, ..., m
    P = Partitions(C,max_part = m).list()
    #print(P)
        
    #Creating all permutations of the integer partitions
    for BasePart in P:
        MoreParts = Permutations(BasePart) 
        # Adding exponents
        for x in MoreParts:
            # The exponent constraint is e = 0
            AllPotentialSides.append([0,x])
                
       
    # Filtered Sides is intialized as a copy of AllPotentialSides.
    # We will remove sides from FilteredSides when the are caught by a filter.
    FilteredSides = []
    for i in AllPotentialSides:
        FilteredSides.append(i)
    
    for L in AllPotentialSides:

        #print("\n L = ",L)

        #Remove any instances where a0 = C
        if L[1][0] == C:
            FilteredSides.remove(L)
            #print("This L got filtered since the first square equals the center (unilaterality check)")
            continue
            
        #Remove any instances where L has ai = a(i+1) for some i = 0, ..., n-2
        flag = False
        for i in range(len(L[1])-1):
            if L[1][i] == L[1][i+1]:
                flag = True
                break
        if flag == True:
            FilteredSides.remove(L)
            #print("This L got filtered since the consecutive sides are equal")
            continue

        #Remove any instances where L contains "a,1,b" or "a,2,b" since any such instance
        #violates the vortex condition for size-1 and size-2 squares
        flag = False
        for i in range(1,len(L[1])-1):
            if (L[1][i] == 1) or (L[1][i] == 2):
                flag = True
                break
        if flag == True:
            FilteredSides.remove(L)
            #print("This L got filtered since there was a 1 or 2 in the middle")
            continue
    
    #print(FilteredSides)
    
    return(FilteredSides)

In [74]:
Side_Type_iii_Generator(7,5)

[[0, [5, 2]],
 [0, [2, 5]],
 [0, [1, 5, 1]],
 [0, [4, 3]],
 [0, [3, 4]],
 [0, [2, 4, 1]],
 [0, [1, 4, 2]],
 [0, [2, 3, 2]]]

## Side Type iv

In [64]:
# Side type iv (doublehang)
# Type iv has the following constraints:

# exponent constrait: 0 < e <= a0. In certain special cases e may be a continuous parameter,
# but most times it must be an integer due to geometric constraints

# partition constraint:
# Both a0 and ak must overhang.
# The overhang of a0 can be as much as m.
# The overhang of ak can be as much as m-1.
# C + 2 <= sum(ai,(i,0,k)) <= (C + 2m - 1)
# This tells us that we want to find integer partitions of 
# C+2, C+3, ..., and (C + 2m - 1) to get the side sequences a0,a1,...,ak 

# Overlap constraints
# The last square ak must overhang but not align with the corner.
# This gives rise to the following:
# sum(ai,(i,0,k-1)) - e < C < sum(ai,(i,0,k)) - e
# This will serve as a filter on partitions together with e.

def Side_Type_iv_Generator(C,m):

    # Using integer partitions to create a list of all potential sides of the form [e,[a0,a1,...,ak]]
    # The partition constraint tells us to find partitions of C+2, C+3, ..., and (C + 2m - 1)
    AllPotentialSides = []
    for n in range(C + 2 , C + 2*m):
        
        #Generate all partitions of n using integers 1, 2, ..., m
        P = Partitions(n,max_part = m).list()
        #print(P)
        
        #Creating all permutations of the integer partitions
        for BasePart in P:
            MoreParts = Permutations(BasePart) 
            
            # Adding integer exponents
            # Note: Later need to analyze which partitions should have a continuous parameter for the exponent.
            for x in MoreParts:
                # The exponent constraint is 0 < e <= a0 and e is an integer
                a0 = x[0]
                for e in range(1,a0+1):
                    AllPotentialSides.append([e,x])
                
       
    # Filtered Sides is intialized as a copy of AllPotentialSides.
    # We will remove sides from FilteredSides when the are caught by a filter.
    FilteredSides = []
    for i in AllPotentialSides:
        FilteredSides.append(i)
    
    for L in AllPotentialSides:

        #print("\n L = ",L)
            
        #Remove any instances where L has ai = a(i+1) for some i = 0, ..., n-2
        flag = False
        for i in range(len(L[1])-1):
            if L[1][i] == L[1][i+1]:
                flag = True
                break
        if flag == True:
            FilteredSides.remove(L)
            #print("This L got filtered since the consecutive sides are equal")
            continue

        #Remove any instances where L contains "a,1,b" or "a,2,b" since any such instance
        #violates the vortex condition for size-1 and size-2 squares
        flag = False
        for i in range(1,len(L[1])-1):
            if (L[1][i] == 1) or (L[1][i] == 2):
                flag = True
                break
        if flag == True:
            FilteredSides.remove(L)
            #print("This L got filtered since there was a 1 or 2 in the middle")
            continue

        # Using the overlap constraints to filter out partitions.
        # Constraint: sum(ai,(i,0,k-1)) - e < C < sum(ai,(i,0,k)) - e
        # Remove any L that does not satisfy these conditions
        
        Lsum1 = sum(L[1][i] for i in range(0,len(L[1])-1))
        Lsum2 = sum(L[1][i] for i in range(len(L[1])))
        #print("Lsum1 = ", Lsum1)
        #print("Lsum2 = ", Lsum2)

        # Checking overlap constraint
        if (Lsum1 - L[0]) >= C or C >= (Lsum2 - L[0]):
            FilteredSides.remove(L)
            #print("This L got filtered by overlap constraint #2")
            continue
        #else:
        #    print("This L survived")
        #    print("C = ", C)
        #    print("Lsum1 = ", Lsum1)
        #    print("Lsum2 = ", Lsum2)
    
    #print(FilteredSides)
    
    return(FilteredSides)

In [65]:
Side_Type_iv_Generator(7,5)

[[1, [5, 4]],
 [1, [4, 5]],
 [1, [1, 5, 3]],
 [1, [1, 3, 5]],
 [1, [2, 5, 2]],
 [1, [4, 3, 2]],
 [1, [3, 4, 2]],
 [1, [2, 4, 3]],
 [1, [2, 3, 4]],
 [1, [1, 5, 4]],
 [1, [1, 4, 5]],
 [2, [5, 3, 2]],
 [2, [3, 5, 2]],
 [1, [2, 5, 3]],
 [2, [2, 5, 3]],
 [1, [2, 3, 5]],
 [2, [2, 3, 5]],
 [1, [3, 4, 3]],
 [2, [3, 4, 3]],
 [3, [5, 4, 2]],
 [3, [4, 5, 2]],
 [1, [2, 5, 4]],
 [2, [2, 5, 4]],
 [1, [2, 4, 5]],
 [2, [2, 4, 5]],
 [2, [3, 5, 3]],
 [3, [3, 5, 3]],
 [1, [4, 3, 4]],
 [2, [4, 3, 4]],
 [3, [4, 3, 4]],
 [3, [5, 4, 3]],
 [4, [5, 4, 3]],
 [2, [5, 3, 4]],
 [3, [5, 3, 4]],
 [4, [5, 3, 4]],
 [3, [4, 5, 3]],
 [4, [4, 5, 3]],
 [1, [4, 3, 5]],
 [2, [4, 3, 5]],
 [3, [4, 3, 5]],
 [4, [4, 3, 5]],
 [2, [3, 5, 4]],
 [3, [3, 5, 4]],
 [1, [3, 4, 5]],
 [2, [3, 4, 5]],
 [3, [3, 4, 5]],
 [2, [5, 3, 5]],
 [3, [5, 3, 5]],
 [4, [5, 3, 5]],
 [5, [5, 3, 5]],
 [3, [4, 5, 4]],
 [4, [4, 5, 4]],
 [3, [5, 4, 5]],
 [4, [5, 4, 5]],
 [5, [5, 4, 5]]]